<a href="https://colab.research.google.com/github/alzimna/Anmul/blob/main/Chapters/Bab_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [258]:
import numpy as np
import pandas as pd
import scipy
import os
import math
import cv2
import shutil
import requests, zipfile, io
from PIL import Image
from skimage import feature, color, data
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from IPython.display import clear_output
from numpy.linalg import inv as inv
from numpy.linalg import eig as eig
from numpy import transpose as t
from numpy.linalg import det as det

# **Example 10.4.2**

In [259]:
url = 'https://github.com/alzimna/Anmul/raw/main/data/t10.1.xlsx'
data = pd.read_excel(url)
print(data.head())

   Obs    y1    y2    y3   x1    x2   x3
0    1  41.5  45.9  11.2  162  23.0  3.0
1    2  33.8  53.3  11.2  162  23.0  8.0
2    3  27.7  57.5  12.7  162  30.0  5.0
3    4  21.7  58.8  16.0  162  30.0  8.0
4    5  19.9  60.6  16.2  172  25.0  5.0


In [260]:
n = data.shape[0]
one = pd.DataFrame(np.ones(n),columns = ["x0"])
X = pd.concat([one,data.iloc[:,4:7]],axis = 1)
print("matriks X =")
print(X.head())
Y = data.iloc[:,1:4]
print("matriks Y =")
print(Y.head())
B = inv(t(X) @ X) @ t(X) @ Y
print("Diperoleh matriks B =")
print(round(B,2))

matriks X =
    x0   x1    x2   x3
0  1.0  162  23.0  3.0
1  1.0  162  23.0  8.0
2  1.0  162  30.0  5.0
3  1.0  162  30.0  8.0
4  1.0  172  25.0  5.0
matriks Y =
     y1    y2    y3
0  41.5  45.9  11.2
1  33.8  53.3  11.2
2  27.7  57.5  12.7
3  21.7  58.8  16.0
4  19.9  60.6  16.2
Diperoleh matriks B =
       y1     y2      y3
0  332.11 -26.04 -164.08
1   -1.55   0.40    0.91
2   -1.42   0.29    0.90
3   -2.24   1.03    1.15


# **Example 10.5.1**

In [261]:
url = 'https://github.com/alzimna/Anmul/raw/main/data/t10.1.xlsx'
data = pd.read_excel(url)
print(data.head())

   Obs    y1    y2    y3   x1    x2   x3
0    1  41.5  45.9  11.2  162  23.0  3.0
1    2  33.8  53.3  11.2  162  23.0  8.0
2    3  27.7  57.5  12.7  162  30.0  5.0
3    4  21.7  58.8  16.0  162  30.0  8.0
4    5  19.9  60.6  16.2  172  25.0  5.0


In [262]:
X = pd.concat([one,data.iloc[:,4:7]],axis = 1)
Y = data.iloc[:,1:4]
E = (t(Y) @ Y) - (t(B).values @ t(X).values @ Y.values)
H = (t(B).values @ t(X).values @ Y.values) - n * t(ybar) @ ybar
print("Matriks E = ")
print(round(E,3))
print("Matriks H = ")
print(np.matrix.round(H,3))
w,V = eig(inv(E) @ H)
print("Nilai eigen dari inv(E) @ H adalah ")
print(w)

Matriks E = 
        y1       y2       y3
y1  80.174  -21.704  -65.923
y2 -21.704  249.462 -179.496
y3 -65.923 -179.496  231.197
Matriks H = 
[[1707.158 -492.532 -996.584]
 [-492.532  151.002  283.607]
 [-996.584  283.607  583.688]]
Nilai eigen dari inv(E) @ H adalah 
[2.63183492e+01 1.00352667e-01 3.29808380e-03]


In [263]:
# Lambda Wilks Test
Lambda = 1
for item in w :
  Lambda*=1/(1+item)
print("Lambda = ",Lambda)

# Roys test
theta = max(w)/(1+max(w))
print("Theta = ",theta)

# Pillai
V = 0
for item in w :
  V+=item/(1+item)
print("V = ",V)

# Lawley
U = 0
for item in w :
  U+=item
print("U = ",U)

Lambda =  0.033157643997862495
Theta =  0.9633945670691759
V =  1.0578822671556265
U =  26.42199996668044


# **Example 10.5.2**

In [264]:
Sec = pd.DataFrame()
Sec["x1^2"] = X.iloc[:,1]**2
Sec["x2^2"] = X.iloc[:,2]**2
Sec["x3^2"] = X.iloc[:,3]**2
Sec["x1x2"] = X.iloc[:,1] * X.iloc[:,2]
Sec["x1x3"] = X.iloc[:,1] * X.iloc[:,3]
Sec["x2x3"] = X.iloc[:,2] * X.iloc[:,3]
X = pd.concat([X,Sec],axis = 1)
Y = data.iloc[:,1:4]
B = inv(t(X) @ X) @ t(X) @ Y
E = (t(Y) @ Y) - (t(B).values @ t(X).values @ Y.values)
H = (t(B).values @ t(X).values @ Y.values) - n * t(ybar) @ ybar
print("Matriks E = ")
print(round(E,3))
print("Matriks H = ")
print(np.matrix.round(H,3))
w,V = eig(inv(E) @ H)
print("Nilai eigen dari inv(E) @ H adalah ")
print(w)

Matriks E = 
        y1      y2       y3
y1  46.208   9.643  -57.179
y2   9.643  60.675  -76.025
y3 -57.179 -76.025  146.179
Matriks H = 
[[ 1741.123  -523.878 -1005.327]
 [ -523.878   339.789   180.136]
 [-1005.327   180.136   668.706]]
Nilai eigen dari inv(E) @ H adalah 
[48.90845676  0.27212976  9.84592745]


In [265]:
Lambda_r = Lambda
# Lambda Wilks Test
Lambda_f = 1
for item in w :
  Lambda_f*=1/(1+item)
print("Lambda = ",Lambda_f/Lambda_r)

Lambda =  0.043796972229939696


# **Example 10.6**

In [266]:
S = data.iloc[:,1:].cov()

In [267]:
S

,y1,y2,y3,x1,x2,x3
y1,99.296199,-28.568626,-59.028129,-41.948538,-9.487573,-7.363889
y2,-28.568626,22.248012,5.783947,11.848830,1.598392,3.025000
y3,-59.028129,5.783947,45.271404,24.138596,6.430848,3.972222
x1,-41.948538,11.848830,24.138596,38.672515,-12.172515,-0.222222
x2,-9.487573,1.598392,6.430848,-12.172515,17.950292,1.222222
x3,-7.363889,3.025000,3.972222,-0.222222,1.222222,2.666667


In [268]:
Syy = S.iloc[:3,:3]
Syx = S.iloc[:3,3:]
Sxy = S.iloc[3:,:3]
Sxx = S.iloc[3:,3:]
RM_squared = det(Syx.values @ inv(Sxx) @ Sxy.values)/det(Syy)
print("RM_squared = ",round(RM_squared,5))
RV = np.trace(Sxy.values @ Syx.values)/np.sqrt(np.trace(Sxx.values @ Sxx.values) * np.trace(Syy.values @ Syy.values))
print("RV = ",round(RV,3))


RM_squared =  0.00029
RV =  0.403
